In [4]:
import numpy as np
import pandas as pd
import time
from rlberry.agents import AgentWithSimplePolicy
from rlberry_scool.agents import UCBVIAgent
from rlberry_research.envs.benchmarks.grid_exploration.nroom import NRoom
from rlberry.manager import(
    ExperimentManager,
    evaluate_agents,
    plot_writer_data,
    read_writer_data,
)

In [7]:
# define the environment GridWorld NRoom
env=NRoom(
    nrooms=9,
    remove_walls=False,
    room_size=9,
    initial_state_distribution="center", #see if we can change to others
    include_traps=True,
)
horizon=env.observation_space.n
T=10000 #number of episodes to play
agent=UCBVIAgent(env,gamma=0.999,horizon=horizon)
print("fitting")
info=agent.fit(budget=T)
print(info)

fitting


[INFO] 10:47:        agent_name  worker  episode_rewards  max_global_step
                       UCBVI       -1         0.01              1 
[INFO] 10:52: [UCBVI[worker: -1]] | max_global_step = 2 | episode_rewards = 0.01 | n_visited_states = 16 |  
[INFO] 10:57: [UCBVI[worker: -1]] | max_global_step = 3 | episode_rewards = 0.03 | n_visited_states = 20 |  
[INFO] 11:03: [UCBVI[worker: -1]] | max_global_step = 4 | episode_rewards = 0.04 | n_visited_states = 33 |  
[INFO] 11:08: [UCBVI[worker: -1]] | max_global_step = 5 | episode_rewards = 0.01 | n_visited_states = 36 |  
[INFO] 11:13: [UCBVI[worker: -1]] | max_global_step = 6 | episode_rewards = 0.01 | n_visited_states = 41 |  
[INFO] 11:18: [UCBVI[worker: -1]] | max_global_step = 7 | episode_rewards = 0.01 | n_visited_states = 46 |  
[INFO] 11:23: [UCBVI[worker: -1]] | max_global_step = 8 | episode_rewards = 0.02 | n_visited_states = 49 |  
[INFO] 11:28: [UCBVI[worker: -1]] | max_global_step = 9 | episode_rewards = 0.02 | n_visited_sta

In [ ]:
env.enable_rendering()

for _ in range(10):
    observation,info=env.reset()
    for h in range(horizon):
        action=agent.policy(observation)
        observation,reward,terminated,truncated,info=env.step(action)
        done=terminated or truncated
        if done:
            break
env.render()
video=env.save_vedio("_video/video_plot_rooms.mp4")

In [ ]:
# define BPI-UCBVI agents
from rlberry.agents import AgentWithSimplePolicy
from rlberry.utils.writers import DefaultWriter
class AdaptiveAgent(AgentWithSimplePolicy):
    name="BPIUCB"

    def __init__(
            self,
            env,
            discount_factor=0.99,
            horizon=100,
            **kwargs
    ):
        super().__init__(env=env,**kwargs)

        #check environment
        assert isinstance(self.env.observation_space, spaces.Discrete)
        assert isinstance(self.env.action_space, spaces.Discrete)

        S=env.observation_space.n
        A=env.action_space.n
        self.gamma=discount_factor #gamma
        self.horizon=horizon
        # other checks
        assert self.gamma >= 0 and self.gamma <= 1.0
        if self.horizon is None:
            assert self.gamma < 1.0, "If no horizon is given, gamma must be smaller than 1."
            self.horizon = int(np.ceil(1.0 / (1.0 - self.gamma)))

        self.P_hat=np.ones((S,A,S))*1.0/S #estimated transition kernel
        self.visitation_count=np.zeros((S,A,S)) #visitation count

        def fit(self,budget,**kwargs):
            """
            The parameter budget can represent the number of steps, the number of episodes etc,
            depending on the agent.
            * Interact with the environment (self.env);
            * Train the agent
            * Return useful information
            """
            T=budget
            rewards=np.zeros(T)
            for t in range(T):
                observation,info=self.env.reset()
                done=False
                while not done:
                    action=self.policy(observation)
                    next_step,reward,terminated,truncated,info=self.env.step(action)
                    #update visitation count and policy
                    self.update(observation,action,next_step)
                    observation=next_step
                    done=terminated or truncated
                    rewards[t]+=reward
            info={"episode_rewards": rewards}
            return info
        
        def eval(self,**kwargs):
            """
            Returns a value corresponding to the evaluation of the agent on the
            evaluation environment.

            For instance, it can be a Monte-Carlo evaluation of the policy learned in fit().
            """
            return super().eval() #use the eval() from AgentWithSimplePolicy
        
        def policy(self,observation,explo=True):

        

